In [ ]:
import concurrent.futures
import os
from functools import partial

import numpy as np

from safe_module import distance

In [ ]:
write = False

In [ ]:
study = "MSAT"
file_path = "../data/TX2008.V1.P010.dat"
data = np.loadtxt(file_path)
n_material = np.size(data,0)
lons = data[:n_material,1]
lats = data[:n_material,0]
cijs = data[:n_material,3:24]

In [ ]:
symmetry_classes = ["ISO", "XISO", "TET", "ORTH", "MONO"]

n_symmetry_classes = len(symmetry_classes)

In [ ]:
betas = np.empty((n_symmetry_classes, n_material))
thetas = np.empty((n_symmetry_classes, n_material))
sigmas = np.empty((n_symmetry_classes, n_material))
phis = np.empty((n_symmetry_classes, n_material))

In [ ]:
num_workers = os.cpu_count()
tracker = np.arange(n_material)
c_vecs = [cijs[i,:] for i in range(n_material)]

for i, sigma in enumerate(symmetry_classes):

    print(f'running minimizer for symmetry class: {sigma}')

    distance_new = partial(distance, use_parallel_processing=False)
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = list(executor.map(distance_new, c_vecs, [sigma]*n_material, tracker))

    for j in range(n_material):
        betas[i,j], thetas[i,j], sigmas[i,j], phis[i,j] = futures[j]

In [ ]:
if write:
    np.save(f'betas_{study}', betas)
    np.save(f'thetas_{study}', thetas)
    np.save(f'sigmas_{study}', sigmas)
    np.save(f'phis_{study}', phis)